## Lunar eclpise code

In [21]:
%time

import csv
from skyfield.api import load, Topos, wgs84, N, S, W, E
from skyfield.positionlib import ICRF

#CONSTANTS
# N,S,W,E are skyfield constants to put the negative sign for lat/Lon as needed
#Default coordinates are for NYC
LOC_NAME = 'nyc' #str constant for file naming
MY_LAT = 40.7128 *N
MY_LON = 74.0060 *W  
MY_ELEV_M = 10 #in meters


# We need the position of celestial bodies.. so we need to get ephemeerides from JPL =O
# https://ssd.jpl.nasa.gov/?planet_eph_export
# DE440 : Created June 2020; compared to DE430, about 7 years of new data have
#        beend added.
#        Referred to the International Celestial Reference Frame version 3.0.
#        Covers JED 2287184.5, (1549 DEC 31) to JED 2688976.5, (2650 JAN 25).

# DE441 : Created June 2020; compared to DE431, about 7 years of new data have 
#        been added.
#        Referred to the International Celestial Reference Frame version 3.0.
#        Covers JED -3100015.5, (-13200 AUG 15) to JED 8000016.50, (17191 MAR 15).

#        DE440 and DE441 are documented in the following document:
#        https://doi.org/10.3847/1538-3881/abd414
#        (NOTE: this paper has been accepted for publication in December, 2020;
#         this link will become available sometime in January)
        
# https://rhodesmill.org/skyfield/planets.html#popular-ephemerides
# de422.bsp is only 17mb and goes from 1900-2050 (issued 2008-02) 150yr
# de430t.bsp is 128mb covering 1550-2650 (issued 2010-02) 1100yr
# de440t.bst is an updated version of de430t w/ 7 years more data

eph = load('de440t.bsp')
ts = load.timescale()
#switch to gregorian calendar from julian at calendar switch date
from skyfield.api import GREGORIAN_START 
ts.julian_calendar_cutoff = GREGORIAN_START 

# Find Lunar eclipses https://rhodesmill.org/skyfield/almanac.html#lunar-eclipses
from skyfield import eclipselib

earth = eph['earth']
sun = eph['sun']
moon = eph['moon']

# NYC lat/lon
place = earth + wgs84.latlon(MY_LAT, MYLON, elevation_m=MY_ELEV_M)

t0 = ts.ut1(1550, 1, 1) 
t1 = ts.ut1(2640, 1, 1)
t, y, details = eclipselib.lunar_eclipses(t0, t1, eph)
# eclipselib.lunar_eclipses:
# * A :class:`~skyfield.timelib.Time` giving the dates of each eclipse.
# * An integer array of codes identifying how complete each eclipse is.
# * A dictionary of further supplementary details about each eclipse.

heading = ('visibility','altitude','date','eclipse_type')
lunar_data = []

writer = csv.writer(open(LOC_NAME+"_lunar_complete.csv",'w'))
writer.writerow(heading)
for ti, yi in zip(t, y):
  alt, az, distance = place.at(ti).observe(moon).apparent().altaz()
  if alt.degrees > 0 :
    vis = 'visible'
  else:
    vis = "not_visible"
  row = (vis, 
         alt.degrees, 
         ti.utc_strftime('%Y-%m-%d'),
         #'y={}'.format(yi),
         eclipselib.LUNAR_ECLIPSES[yi])
  lunar_data.append(row)
  writer.writerow(row)
writer.close()



CPU times: user 10 µs, sys: 8 µs, total: 18 µs
Wall time: 7.87 µs


In [38]:
# Save the CSV out



# Also make a version that only has 'visible' non-penumbra eclipses
# This will be our actual data set
writer = csv.writer(open(LOC_NAME+"_lunar_observed.csv",'w'))
writer.writerow(heading)

for ti, yi in zip(t, y):
  alt, az, distance = place.at(ti).observe(moon).apparent().altaz()
  if alt.degrees > 0 :
    vis = 'visible'
  else:
    vis = "not_visible"
    
  row = (vis, 
         alt.degrees, 
         ti.utc_strftime('%Y-%m-%d'),
         #'y={}'.format(yi),
         eclipselib.LUNAR_ECLIPSES[yi])
  if eclipselib.LUNAR_ECLIPSES[yi] != 'Penumbral' \
    and vis == 'visible':
      writer.writerow(row)
writer.close()

In [22]:
import pandas as pd
df = pd.DataFrame.from_records(lunar_data,columns=heading)

In [25]:
df

,visibility,altitude,date,eclipse_type
0,visible,45.060597,1550-03-03,Penumbral
1,not_visible,-26.531747,1550-04-01,Penumbral
2,visible,42.774474,1550-08-27,Penumbral
3,not_visible,-31.804474,1551-02-20,Total
4,visible,35.646395,1551-08-16,Partial
...,...,...,...,...
2646,not_visible,-14.270010,2637-10-25,Total
2647,not_visible,-45.793073,2638-04-20,Partial
2648,visible,52.948904,2638-10-15,Penumbral
2649,not_visible,-37.515163,2639-03-11,Partial


In [30]:
df.groupby(['visibility','eclipse_type']).count()['date']

visibility   eclipse_type
not_visible  Partial         460
             Penumbral       498
             Total           389
visible      Partial         447
             Penumbral       480
             Total           377
Name: date, dtype: int64

36.697766317211496
